# Clusterização (Modelo Não Supervisionado Fuzzy C Means)
Este notebook explora o algoritmo de aprendizado de máquina não supervisionado Fuzzy C Means. <br>
O objetivo deste método é agrupar dados em diferentes grupos (clusters), em que cada dado pode pertencer a mais de um cluster simultanemante, sem o uso de rótulos prévios, e buscar tendências. </p>

---
## Instalação de bibliotecas
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas==2.1.4 numpy==1.26.4 matplotlib==3.7.5 seaborn==0.13.2 scikit-learn==1.4.2 scikit-fuzzy==0.0.8 plotly==5.24.1

---
## Inicialização de bibliotecas
Importa as bibliotecas necessárias:

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import skfuzzy as fuzz
import plotly.express as px

from skfuzzy import cmeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Configuração para mostrar todas as colunas no pandas
pd.set_option('display.max_columns', None)

# Configuração para exibir os gráficos diretamente no notebook
%matplotlib inline

---
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas:

Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

1. **Obtenha o Arquivo de Dados**:
   - O arquivo `BASE DE SINISTRO UNIPAR BRADESCO.csv` deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

2. **Posicione o Arquivo na Pasta Correta**:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
3. **Verifique o Caminho do Arquivo**:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     ```python
     df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')
     ```
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

4. **Rodar o Notebook**:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

**Nota**: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

&ensp;Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [72]:
df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')

---
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [ ]:
# Limpeza de dados

# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')

df

---
## Codificação
Este trecho de código realiza a codificação de colunas categóricas, transformando-as em valores numéricos para facilitar o uso no modelo<br>
de machine learning. As colunas categóricas são mapeadas para índices numéricos, e o resultado é armazenado em um novo DataFrame.<br>
Além disso, a coluna de data é convertida para o formato 'YYYYMMDD', e as colunas numéricas e de data são adicionadas ao DataFrame final.

In [74]:
# Lista de colunas que serão codificadas (colunas categóricas)
colunas_para_codificar = [
    'Codigo Empresa Sinistro',
    'Sexo Sinistro',
    'Faixa-Etária Nova Sinistro',
    'Descricao Plano Sinistro',
    'Codigo Servico Sinistro',
]

# Colunas numéricas e de data que não precisam de codificação
colunas_nao_codificadas = [
    'Dt Data Sinistro',
    'Valor Pago Sinistro',
]

# Para armazenar as novas colunas codificadas
novas_colunas_codificadas = {}

# Codificação das colunas categóricas
for coluna in colunas_para_codificar:
        unique_sorted_values = sorted(df[coluna].unique())
        df[f'Codificada {coluna}'] = df[coluna].apply(lambda x: unique_sorted_values.index(x))
        # Armazenar as novas colunas codificadas
        novas_colunas_codificadas[f'Codificada {coluna}'] = df[f'Codificada {coluna}']

# Criando um novo DataFrame com as novas colunas codificadas
df_novo = pd.DataFrame(novas_colunas_codificadas)

# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

# Adicionando as colunas numéricas e de data ao DataFrame final
for coluna in colunas_nao_codificadas:
    df_novo[coluna] = df[coluna]
 
df = df_novo

#### Verificação

In [ ]:
# Exibindo as 10 primeiras linhas do novo DataFrame
df.head(10).sort_values(by='Valor Pago Sinistro', ascending=False)

---
## Detecção e Remoção de Outliers
Esse bloco de código remove os outlier da coluna *Valor Pago Sinistro* aplicando o _IsolationForest_ com uma contaminação de 5%, criando um novo DataFrame chamado de `df_clean`. Além disso, ele padroniza as colunas numéricas e as armazena em uma nova variável `numeric_colums` apenas com as colunas numéricas do DataFrame para futuras análises.

In [76]:
# Aplicar IsolationForest para detectar outliers
iso_forest = IsolationForest(contamination=0.05)  # 5% de contaminação, ajuste conforme necessário
outliers = iso_forest.fit_predict(df[['Valor Pago Sinistro']])

# Remover os outliers
df_clean = df[outliers == 1]

# Selecionar apenas as colunas numéricas para padronização
numeric_columns = df_clean.select_dtypes(include=['float64', 'int64']).columns

#### Verificação

In [ ]:
df_clean.describe().round(2) #Arredonda os dados com 2 casa decimais

---
## Tratamento de Dados

Este código padroniza as colunas numéricas no DataFrame `df_clean` após a remoção de outliers, utilizando o `StandardScaler` para ajustar os dados à média 0 e desvio padrão 1. 

A função `fit_transform` é aplicada para transformar as colunas numéricas, e o resultado é armazenado diretamente no DataFrame. Por fim, as primeiras linhas do DataFrame limpo e padronizado são exibidas com `df_clean.head()`.

In [ ]:
# Padronizar os dados após remover os outliers
scaler = StandardScaler()
df_clean.loc[:, numeric_columns] = scaler.fit_transform(df_clean[numeric_columns])  # Usando .loc para evitar o SettingWithCopyWarning

# Mostrar as primeiras linhas do dataframe limpo e padronizado
df_clean.head()

#### Verificação

In [ ]:
df.head(10).sort_values(by='Valor Pago Sinistro', ascending=False)

In [ ]:
df.max()

---
## Conversão e Extração de Componentes de Data

Este código converte a coluna `Dt Data Sinistro` do DataFrame `df` para o formato datetime, utilizando o padrão 'YYYYMMDD'. Em seguida, extrai o componente do mês da data e o armazena em uma nova coluna chamada `Mes`. Se necessário, os valores dessa coluna são padronizados utilizando o `StandardScaler`, com o resultado armazenado na nova coluna `Mes_scaled`. O DataFrame resultante contém as colunas originais e as novas colunas derivadas da data.


In [ ]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de dia
df['Mes'] = df['Dt Data Sinistro'].dt.month

# Padronizar (escalonar) os componentes se necessário
scaler = StandardScaler()

df[["Mes_scaled"]] = scaler.fit_transform(df[["Mes"]])

df.head()

---
## Remoção de Colunas
Este código remove as colunas `Mes` e `Dt Data Sinistro` do DataFrame `df`. A remoção dessas colunas é feita diretamente no DataFrame, utilizando o parâmetro `inplace=True`, o que garante que as alterações sejam aplicadas sem a necessidade de criar uma cópia do DataFrame. O resultado é um DataFrame que não contém mais as colunas especificadas.

In [82]:
df.drop(columns=['Mes'], inplace=True)

df.drop(columns=['Dt Data Sinistro'], inplace=True)

#### Verifcação

In [ ]:
df.head()

---
## Exploração de dados
### Matriz de Disperção e Correlação
Esse bloco gera uma matriz de gráficos de disperção (*scatter plots*) entre cada par de variáveis numéricas presentes no `df_clean`.<br>
Usamos o pairplot para poder ter uma visualização mais rápida e intuitiva das relações entre as variáveis, facilitando a identificação de correlações que fazem mais sentido.

A segunda célula calcula uma matriz de correlação para identificar o grau de influência entre duas features. 

In [84]:
# Habilitar de acordo com a necessidade removendo o símbolo: #

# Verificar a relação entre as variáveis
# sns.pairplot(df_clean) 

In [85]:
# Habilitar de acordo com a necessidade removendo o símbolo: #

# Calcular a matriz de correlação
# sns.heatmap(df_clean.corr(), annot=True, cmap="coolwarm", linewidths=0.5) 

# plt.show()

---
## Modelo Fuzzy C Means
Após tratar e padronizar os dados , a próxima etapa é aplicar o Fuzzy C Means (FCM). Nessa fase, o algoritmo é inicializado com o valor de n (clusters) escolhido.

Esse bloco realiza a aplicação do modelo Fuzzy C-Means para rodar 1 vez com o objetivo de testar o modelo e visualiza, a partir de um gráfico, os clusters resultantes.<br>
Definimos os seguintes hiperparâmetros do modelo:

- n_clusters: Número de clusters a ser formado = 6
- m: Fator de fuzzificação = 1.1, que controla a "suavidade" do pertencimento dos dados aos clusters
- error: Critério de parada do algortimo = 0.01 
- maxiter: Número máximo de iterações = 1000

In [ ]:
# Define o número de clusters
n_clusters = 6

# Fuzzy C-Means clustering
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
    df.T, 6, 1.1, error=0.01, maxiter=1000)

# Atribui os rótulos dos clusters
cluster_labels = np.argmax(u, axis=0)

# Adiciona os rótulos ao dataframe original
df['cluster'] = cluster_labels

# Exibe gráfico para visualização dos clusters
plt.scatter(df['Mes_scaled'], df['Codificada Codigo Servico Sinistro'], c=df['cluster'], cmap='viridis')
plt.scatter(cntr[:, 0], cntr[:, 1], c='red', marker='x')  # Centros dos clusters
plt.title('Fuzzy C-Means Clustering')
plt.xlabel('Mes_scaled')
plt.ylabel('Codificada Codigo Servico Sinistro')
plt.show()

---

### Criação de uma classe 
Esse bloco cria uma classe para o modelo Fuzzy C Means para poder rodar no Grid Search, fornecendo métodos para ajuste (fit), previsão (predict) e cálculo do silhouette score (uma métrica que avalia a qualidade dos clusters), com os hiperparâmetros definidos.

In [87]:
# Classe para Fuzzy C-Means

class FuzzyCMeans(BaseEstimator, ClusterMixin):
    def __init__(self, n_clusters=6, m=1.5, error=0.05, maxiter=300):
        self.n_clusters = n_clusters
        self.m = m
        self.error = error
        self.maxiter = maxiter
        self.labels_ = None
        self.centers_ = None

    def fit(self, X):
        # Fuzzy C-Means clustering
        cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
            X.T, self.n_clusters, self.m, error=self.error, maxiter=self.maxiter
        )
        
        # Obter rótulos de cluster
        self.labels_ = np.argmax(u, axis=0)
        self.centers_ = cntr
        return self

    def predict(self, X):
        return self.labels_

    def silhouette_score(self, X):
        return silhouette_score(X, self.labels_)
    
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    # Evitar erro se apenas 1 cluster for formado
    if len(np.unique(labels)) > 1:
        return silhouette_score(X, labels)
    else:
        return -1

---
## Finetunning de Hiperparâmetro (GridSearch)

Esse bloco prepara os dados e define os parâmetros para realizar uma busca em grade (GridSearch), que tem como objetivo encontrar a melhor combinação de hiperparâmetros para o algoritmo. Inicialmente testamos algumas combinações de parâmetros, mas depois restringimos a somente uma combinação que conseguimos identificar como potencialmente ótimos. O objetivo final é aplicar esses parâmetros ajustados ao modelo e observar a performance.

In [93]:
# Preparar dados para o GridSearch

# Parâmetros ótimos para duas colunas: {'error': 0.05, 'm': 1.5, 'maxiter': 300, 'n_clusters': 7}
X = df[['Codificada Codigo Servico Sinistro', 'Valor Pago Sinistro']].values

# Parâmetros ótimos para df completo: {'error': 0.1, 'm': 1.1, 'maxiter': 1000, 'n_clusters': 6}
X = df.values

# # Define o grid de parâmetros para testar
# param_grid = {
#     'n_clusters': [6, 7],
#     'm': [1.1, 1.5],
#     'error': [0.01, 0.05],
#     'maxiter': [300, 1000]
# }

# Definindo os parâmetros para busca
param_grid = {
    'n_clusters': [6],
    'm': [1.1],
    'error': [0.1],
    'maxiter': [1000]
}

### Análise de Hiperparâmetros com GridSearchCV 

Este código realiza uma busca em grade (GridSearchCV), com uma função de pontuação personalizada para encontrar os melhores parâmetros, e utiliza o silhouette score como métrica de avaliação da qualidade dos clusters (essa célula pode demorar de acordo com os parâmetros testados).

In [ ]:
# Inicializando o GridSearchCV com uma função de pontuação personalizada
grid_search = GridSearchCV(
    estimator=FuzzyCMeans(),
    param_grid=param_grid,
    scoring=silhouette_scorer,  # Usando a função silhouette_score como métrica
    cv=5,
    verbose=3
)

# Executando o GridSearch
grid_search.fit(df)

# Resultados
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor Silhouette Score:", grid_search.best_score_)

### Visualização dos Clusters

Este bloco tem como objetivo visualizar os resultados do agrupamento Fuzzy C-Means utilizando os melhores parâmetros encontrados na busca em grade (GridSearchCV). Ele faz a predição dos clusters, cria um gráfico de dispersão (scatter plot) colorido por cluster e adiciona uma barra de cores com legendas baseadas nos clusters formados.

In [ ]:
# Visualizando os resultados com os melhores parâmetros
best_model = grid_search.best_estimator_
df['cluster'] = best_model.predict(X)

# Visualizando os clusters
scatter = plt.scatter(df['Mes_scaled'],df['Valor Pago Sinistro'], c=df['cluster'], cmap='Spectral', edgecolors='black')
# plt.scatter(cntr[:, 0], cntr[:, 1], c='red', marker='x')  # Centros dos clusters
plt.title('Fuzzy C-Means Clustering com Melhores Parâmetros')
plt.xlabel('Valor Pago Sinistro')
plt.ylabel('Codificada Codigo Servico Sinistro')

# Lista de números de clusters
clusters = np.unique(df['cluster'])

# Criando a legenda com base nos clusters
legend_labels = [f'Cluster {int(cluster)+1}' for cluster in clusters]

# Criando a barra de cores para o gráfico
cbar = plt.colorbar(scatter)
cbar.set_ticks(clusters)
cbar.set_ticklabels(legend_labels)

plt.show()

### Identificação dos dados de um único cluster

Esse bloco filtra o DataFrame para incluir apenas as linhas onde o valor na coluna **cluster** seja igual a 1, ou seja, isso irá mostrar apenas os dados que pertencem ao cluster 1.


In [ ]:
cluster_1_data = df[df['cluster'] == 1]

cluster_1_data

### Gráfico do cluster 1 (baseado na faixa etária)

Esse bloco realiza uma análise e visualização da distribuição de faixas etárias para os dados que pertencem ao cluster 1.<br>
É realizado a contagem da frequência com que cada faixa etária, representada pela coluna 'Codificada Faixa-Etária Nova Sinistro', aparece no cluster 1.




In [ ]:
cluster_1_data = df[df['cluster'] == 1] 

service_counts = cluster_1_data['Codificada Faixa-Etária Nova Sinistro'].value_counts()

# Plotando o gráfico de barras
plt.bar(service_counts.index, service_counts.values)

# Título e rótulos dos eixos
plt.title('Distribuição de Faixa-Etária - Cluster 1')
plt.xlabel('Faixa-Etária')
plt.ylabel('Frequência')

# Exibindo o gráfico
plt.show()



### Gráficos de distribuição de Faixa Etária em cada cluster

Gera uma visualização gráfica da distribuição de faixas etárias em todos os 6 clusters, com base na coluna 'Codificada Faixa-Etária Nova Sinistro'. Usamos essa visualização para analisar como as faixas etárias estão distribuídas em cada cluster que o modelo fez e possivelmente tirar algum *insight* relevante.

In [ ]:
# Definir número de clusters (ajustar de acordo com os dados)
num_clusters = 6

# Obter todas as categorias únicas de "Codificada Faixa-Etária Nova Sinistro"
faixas_etarias = df['Codificada Faixa-Etária Nova Sinistro'].unique()

# Criar um colormap fixo associando cada faixa-etária a uma cor
cmap = plt.get_cmap('tab10')  # Colormap com 10 cores distintas, pode-se usar outros como 'viridis', 'Set3', etc.
color_mapping = {faixa: cmap(i % 10) for i, faixa in enumerate(faixas_etarias)}  # Mapeamento faixa-etária -> cor

# Criar uma figura com subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))  # 2 linhas e 3 colunas de gráficos

# Iterar sobre cada cluster
for i in range(num_clusters):
    cluster_data = df[df['cluster'] == i]  # Filtrar os dados do cluster atual
    service_counts = cluster_data['Codificada Faixa-Etária Nova Sinistro'].value_counts()  # Contar os valores
    
    # Selecionar o eixo correto (usar i//3 para linha e i%3 para coluna)
    ax = axes[i // 3, i % 3]
    
    # Gerar cores para as barras com base na faixa-etária
    bar_colors = [color_mapping[faixa] for faixa in service_counts.index]
    
    # Plotar o gráfico de barras no eixo correspondente
    ax.bar(service_counts.index, service_counts.values, color=bar_colors)
    ax.set_title(f'Distribuição de Faixa Etária por Cluster {i+1}')
    ax.set_xlabel('Faixa Etária')
    ax.set_ylabel('Frequência')

# Adicionar legenda com as faixas-etárias e suas cores
handles = [plt.Rectangle((0,0),1,1, color=color_mapping[faixa]) for faixa in faixas_etarias]
labels = faixas_etarias
fig.legend(handles, labels, title="Faixa Etária", loc='upper center', ncol=len(faixas_etarias))

# Ajustar layout para evitar sobreposição de textos
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Deixar espaço no topo para a legenda

# Exibir os gráficos
plt.show()


---
# Cálculo de métricas

Aqui é realizado o cálculo das métricas escolhidas: Silhouette Score, Davis-Bouldin Index e Calinski-Harabasz Index. Elas ajudam a verificar se a formação dos *clusters* foi bem-sucedida, com base na qualidade da segmentação obtida. O resultado dessas métricas sozinho não deve ser a única maneira de avaliar se o modelo de fato é o melhor, estamos levando em consideração como os *clusters* foram segmentados pelo algoritmo para ter certeza de que as divisões feitas realmente fazem sentido para o projeto.


In [ ]:
# Define os dados pra o grid search e os parametros 
# Preparar dados para clustering
X = df[['Valor Pago Sinistro', 'Codificada Codigo Servico Sinistro']].values

# Inicializando o modelo Fuzzy C-Means com os parâmetros fornecidos
fcm = FuzzyCMeans(n_clusters=6, m=1.1, error=0.05, maxiter=1000)
fcm.fit(X) 

# Adicionando rótulos ao DataFrame
df['cluster'] = fcm.predict(X)

# Calculando o Silhouette Score
silhouette_avg = silhouette_score(X, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

# Calculando o Davies-Bouldin Index
dbi = davies_bouldin_score(X, df['cluster'])
print(f"Davies-Bouldin Index: {dbi}")

# Calculando o Calinski-Harabasz Index
chi = calinski_harabasz_score(X, df['cluster'])
print(f"Calinski-Harabasz Index: {chi}")

---
## Outras análises

### Gráfico 3D 

Esse bloco gera uma visualização dos resultados do modelo Fuzzy C Means, em três dimensões, com base nos melhores parâmetros encontrados pelo GridSearch (GridSearchCV). Ele cria um gráfico tridimensional, onde as variáveis são mapeadas para os eixos X, Y e Z, e os pontos são coloridos de acordo com o cluster ao qual pertencem. As variáveis escolhidas foram as colunas `Codificada Codigo Servico Sinistro` (eixo X), `Valor Pago Sinistro` (eixo Y) e `Codificada Faixa-Etária Nova Sinistro` (eixo Z).


In [ ]:
# Visualizando os resultados com os melhores parâmetros
best_model = grid_search.best_estimator_
df['cluster'] = best_model.predict(X)

# Definindo as variáveis para o gráfico 3D
x = df['Codificada Codigo Servico Sinistro']
y = df['Valor Pago Sinistro']
z = df['Codificada Faixa-Etária Nova Sinistro']  # Use a variável que faz sentido para o z

# Criando a figura 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotando os pontos
scatter = ax.scatter(x, y, z, c=df['cluster'], cmap='Spectral', edgecolors='black')

# Adicionando título e rótulos aos eixos
ax.set_title('Fuzzy C-Means Clustering com Melhores Parâmetros')
ax.set_xlabel('Codificada Codigo Servico Sinistro')
ax.set_ylabel('Valor Pago Sinistro')
ax.set_zlabel('Codificada Faixa-Etária Nova Sinistro')

# Criando a barra de cores para o gráfico
cbar = plt.colorbar(scatter)
clusters = np.unique(df['cluster'])
cbar.set_ticks(clusters)
cbar.set_ticklabels([f'Cluster {int(cluster)+1}' for cluster in clusters])

# Exibindo o gráfico
plt.show()

### Gráfico 3D Interativo

Este código utiliza a biblioteca Plotly Express para criar um gráfico 3D interativo, que visualiza os resultados do modelo Fuzzy C-Means após a busca dos melhores parâmetros por meio do GridSearchCV. Utilizamos as mesmas 3 variáveis do gráfico anterior. Escolhemos criar esses gráficos para ajudar na compreensão das divisões dos cluster e tentar extrair alguns *insights* que possam ser relevantes para nosso projeto.

In [ ]:
# Visualizando os resultados com os melhores parâmetros
best_model = grid_search.best_estimator_
df['cluster'] = best_model.predict(X)

# Criando o gráfico 3D interativo
fig_3d = px.scatter_3d(
    df, 
    x='Codificada Codigo Servico Sinistro', 
    y='Valor Pago Sinistro', 
    z='Codificada Faixa-Etária Nova Sinistro',
    color='cluster',
    title='Fuzzy C-Means Clustering com Melhores Parâmetros',
    color_continuous_scale='Spectral',
    
)

# Atualizando os rótulos dos eixos e o tamanho dos marcadores
fig_3d.update_traces(marker=dict(size=4))  # Reduzindo o tamanho dos pontos

# Atualizando o layout para aumentar o tamanho do gráfico
fig_3d.update_layout(
    scene=dict(
        xaxis_title='Codificada Codigo Servico Sinistro',
        yaxis_title='Valor Pago Sinistro',
        zaxis_title='Codificada Faixa-Etária Nova Sinistro'
    ),
    width=1200,  # Largura do gráfico
    height=800   # Altura do gráfico
)

max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
mid_x = (x.max()+x.min()) * 0.5
mid_y = (y.max()+y.min()) * 0.5
mid_z = (z.max()+z.min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

# Mostrando o gráfico
fig_3d.show()
